Import necessary libraries

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [9]:
# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
# Set random seed for reproducibility
torch.manual_seed(42)

# Define transforms for data augmentation and normalization
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [11]:
# Load the dataset
train_dataset = datasets.ImageFolder('data/train', transform=train_transform)
test_dataset = datasets.ImageFolder('data/test', transform=test_transform)

# Define the CNN model
class CancerCNN(nn.Module):
    def __init__(self):
        super(CancerCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(256 * 28 * 28, 512)
        self.fc2 = nn.Linear(512, 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 256 * 28 * 28)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


Define the loss function and optimizer

In [12]:
model = CancerCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [13]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

Train The model

In [14]:

for epoch in range(10):
    train_loss = 0
    train_correct = 0
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        train_correct += pred.eq(target.view_as(pred)).sum().item()
        loss.backward()
        optimizer.step()

    train_loss /= len(train_loader.dataset)
    train_acc = 100. * train_correct / len(train_loader.dataset)
    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)'.format(
        train_loss, train_correct, len(train_loader.dataset), train_acc))

    # Test the model
    model.eval()
    test_loss = 0
    test_correct = 0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            test_correct += pred.eq(target.view_as(pred)).sum().item()
            all_preds.extend(pred.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    test_loss /= len(test_loader.dataset)
    test_acc = 100. * test_correct / len(test_loader.dataset)
    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, test_correct, len(test_loader.dataset), test_acc))

    # Calculate F1 score
    all_preds = np.array(all_preds)
    all_targets = np.array(all_targets)
    f1_score = sklearn.metrics.f1_score(all_targets, all_preds, average='macro')
    print('F1 score: {:.4f}'.format(f1_score))

# Save the model
torch.save(model.state_dict(), 'cancer_cnn.pth')

Train set: Average loss: 0.0227, Accuracy: 105/178 (59%)
Test set: Average loss: 0.0284, Accuracy: 59/130 (45%)

F1 score: 0.3122
Train set: Average loss: 0.0220, Accuracy: 117/178 (66%)
Test set: Average loss: 0.0241, Accuracy: 92/130 (71%)

F1 score: 0.7076
Train set: Average loss: 0.0200, Accuracy: 130/178 (73%)
Test set: Average loss: 0.0223, Accuracy: 82/130 (63%)

F1 score: 0.5308
Train set: Average loss: 0.0179, Accuracy: 130/178 (73%)
Test set: Average loss: 0.0203, Accuracy: 99/130 (76%)

F1 score: 0.7612
Train set: Average loss: 0.0145, Accuracy: 150/178 (84%)
Test set: Average loss: 0.0239, Accuracy: 87/130 (67%)

F1 score: 0.6585
Train set: Average loss: 0.0160, Accuracy: 138/178 (78%)
Test set: Average loss: 0.0165, Accuracy: 102/130 (78%)

F1 score: 0.7813
Train set: Average loss: 0.0146, Accuracy: 146/178 (82%)
Test set: Average loss: 0.0160, Accuracy: 103/130 (79%)

F1 score: 0.7917
Train set: Average loss: 0.0110, Accuracy: 159/178 (89%)
Test set: Average loss: 0.0347,